<a href="https://colab.research.google.com/github/wonjae11/AI_TEST/blob/main/machinelearning_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mediapipe opencv-python tensorflow ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 62.3 MB/s eta 0:00:00


In [6]:
!pip install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 39.0 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# 데이터 로드
train_df = pd.read_csv('/content/drive/MyDrive/sign_mnist_train.csv', on_bad_lines='skip')
test_df = pd.read_csv('/content/drive/MyDrive/sign_mnist_test.csv', on_bad_lines='skip')


# 레이블과 픽셀 분리
train_labels = train_df['label']
train_data = train_df.drop('label', axis=1)

test_labels = test_df['label']
test_data = test_df.drop('label', axis=1)

# 데이터 정규화
train_data = train_data / 255.0
test_data = test_data / 255.0

# 데이터 형상 변환
train_data = train_data.values.reshape(-1, 28, 28, 1)
test_data = test_data.values.reshape(-1, 28, 28, 1)

# 레이블 원-핫 인코딩
train_labels = to_categorical(train_labels, num_classes=26)
test_labels = to_categorical(test_labels, num_classes=26)


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# 모델 정의
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(26, activation='softmax')
])

# 모델 컴파일
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 모델 학습
model.fit(train_data, train_labels, epochs=10, validation_data=(test_data, test_labels))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
858/858 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.3079 - loss: 2.2714 - val_accuracy: 0.8217 - val_loss: 0.5493
Epoch 2/10
858/858 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7806 - loss: 0.6293 - val_accuracy: 0.8874 - val_loss: 0.3427
Epoch 3/10
858/858 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8819 - loss: 0.3406 - val_accuracy: 0.8995 - val_loss: 0.2903
Epoch 4/10
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9198 - loss: 0.2286 - val_accuracy: 0.9254 - val_loss: 0.2316
Epoch 5/10
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9351 - loss: 0.1824 - val_accuracy: 0.9249 - val_loss: 0.2355
Epoch 6/10
858/858 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9504 - loss: 0.1358 - val_accuracy: 0.9216 - val_loss: 0.2526
Epoch 7/10
858/858 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9562 - loss: 0.1237 - val_accuracy: 0.9350 - val_loss: 0.2185
Epoch 8/10
858/858 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9618 - loss: 0.1077 - val_accuracy: 0

In [19]:
import cv2
import mediapipe as mp
from google.colab.patches import cv2_imshow
from google.colab.output import eval_js
import base64
from PIL import Image
from io import BytesIO
import numpy as np

# MediaPipe 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)

# 웹캠 캡처 UI
def capture_image():
    js = '''
        async function capture() {
            const video = document.createElement('video');
            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            video.srcObject = stream;
            await video.play();

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            const img = canvas.toDataURL('image/jpeg');
            stream.getTracks().forEach(track => track.stop());
            return img;
        }
        capture();
    '''
    data = eval_js(js)
    return data

# 이미지 처리 및 예측
def process_image(data):
    header, encoded = data.split(",", 1)
    image_data = base64.b64decode(encoded)
    image = Image.open(BytesIO(image_data)).convert('L')
    image = image.resize((28, 28))
    image = np.array(image) / 255.0
    image = image.reshape(1, 28, 28, 1)

    # 예측
    prediction = model.predict(image)
    predicted_label = np.argmax(prediction)
    print(f"인식된 수화: {chr(predicted_label + ord('A'))}")

# 실행 예제
data = capture_image()
process_image(data)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
인식된 수화: Q
